In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import os.path as path
import numpy as np
import pandas as pd
import json,re

import torch.utils.data
from glob import glob

import import_ipynb
import tracker_network.data_sources.data_utils as data_utils
import branch_adapter.regression.regression_inference as regression_inference

os.environ['KMP_DUPLICATE_LIB_OK']='True'

importing Jupyter notebook from D:\WETrak\tracker_network\data_sources\data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_inference.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_model_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_network.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_data.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_cascade_inference.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_network.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data_utils.ipynb
importing Jupyter notebook 

In [2]:
class finger_data_15(torch.utils.data.Dataset):
    def __init__(self, root_path, user_name, split, finger_type_list):
        
        self.user_name = user_name
        self.data_path = path.join(root_path,user_name,split)
#         print("datapath:",path.abspath(self.data_path))
        self.finger_type_list = finger_type_list
        
        self.files = glob(path.join(self.data_path,'*'))
        
        self.emg_data_list = []
        self.leap_data_list = []
        
        for file_path in self.files:
            emg_data = pd.read_csv(path.join(file_path,"emg_filtered.csv"),header=None).to_numpy()
            
            leap_data = json.load(open(path.join(file_path,"leap_data_processed.json"),mode='r', encoding='utf-8'))
            leap_data_temp = data_utils.parse_leap_all_finger(leap_data,self.finger_type_list) # [1000,N,4]
            leap_data = leap_data_temp.reshape(len(leap_data_temp),-1) # [1000,N*4]
                        
            self.emg_data_list.append(emg_data)
            self.leap_data_list.append(leap_data)
    
        # smooth leap motion data
        self.leap_data_list = data_utils.smooth_gt(self.leap_data_list,self.finger_type_list)

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        emg_data = self.emg_data_list[idx]
        leap_data = self.leap_data_list[idx]
           
        emg = np.expand_dims(emg_data,axis=0).astype(np.float32)
        gt = np.expand_dims(leap_data,axis=0).astype(np.float32)

        return emg,gt
    
    def get_test(self, idx):
        return self.__getitem__(idx)

In [3]:
class finger_data_234(torch.utils.data.Dataset):
    def __init__(self, root_path, user_name, split, finger_type_list,classify_num=0,inference_mode=False,output_path=None):
        
        self.data_path = path.join(root_path,user_name,split)
#         print("datapath:",path.abspath(self.data_path))
        self.finger_type_list = finger_type_list
        
        self.files = glob(path.join(self.data_path,'*'))
        self.files.sort(key = self.extract_number)
        
        self.emg_data_list = []
        self.leap_data_list = []
        
        for file_path in self.files:
            emg_data = pd.read_csv(path.join(file_path,"emg_filtered.csv"),header=None).to_numpy()
            
            leap_data = json.load(open(path.join(file_path,"leap_data_processed.json"),mode='r', encoding='utf-8'))
            leap_data_temp = data_utils.parse_leap_all_finger(leap_data,self.finger_type_list) # [1000,N,4]
            leap_data = leap_data_temp.reshape(len(leap_data_temp),-1) # [1000,N*4]
            
            self.emg_data_list.append(emg_data)
            self.leap_data_list.append(leap_data)
        
        # emg data z-score normalize  
        self.emg_data_list = data_utils.emg_z_norm(self.emg_data_list)
        # smooth leap motion data
        self.leap_data_list = data_utils.smooth_gt(self.leap_data_list,self.finger_type_list)
        
        if inference_mode == False:
            # leap motion data min-max normalize 
            min_vals_list,max_vals_list,new_leap_data_list = data_utils.gt_normalize(self.leap_data_list,finger_type_list)
            
            self.min_vals = min_vals_list
            self.max_vals = max_vals_list
            self.leap_data_list= new_leap_data_list
            
        elif inference_mode == True:
            pred_min_vals,pred_max_vals = regression_inference.get_regression_inference(root_path, output_path, user_name,split,finger_type_list,classify_num,inference_mode)

            self.min_vals = pred_min_vals
            self.max_vals = pred_max_vals
            
    def extract_number(self,filename):
        pattern = r'\d+'
        matches = re.findall(pattern, filename)
        number = int(matches[0])
        
        return number
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        emg_data = self.emg_data_list[idx]
        leap_data = self.leap_data_list[idx]
        min_val = self.min_vals[idx]
        max_val = self.max_vals[idx]
        
        emg = np.expand_dims(emg_data,axis=0).astype(np.float32)
        gt = np.expand_dims(leap_data,axis=0).astype(np.float32) #每一个time step的运动角度
        min_val = np.expand_dims(min_val,axis=(0,1)).astype(np.float32) #每一段2s数据的最小值
        max_val = np.expand_dims(max_val,axis=(0,1)).astype(np.float32) #每一段2s数据的最大值

        return emg,gt,min_val,max_val
    
    def get_test(self, idx):
        return self.__getitem__(idx)